<a href="https://colab.research.google.com/github/CamiMM32/Sistema_de_Busca_em_Docs_usando_Embeddings_e_a_Gemini_API/blob/main/sistema_para_busca_em_docs_usando_embeddings_e_a_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instala a biblioteca google-generativeai
!pip install -U -q google-generativeai

In [ ]:
#Importações e configurações iniciais. Importa as bibliotecas necessárias: numpy para cálculos numéricos, pandas para a manipulação de dados e google.generativeai para acessar a API do Google Generative IA
import numpy as np
import pandas as pd
import google.generativeai as genai

#Define a chave da API do Google
GOOGLE_API_KEY = "API_Key"
#Configura a biblioteca com a chave da API
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
#Lista os modelos disponíveis na API que suportam embeddings
for m in genai.list_models():
  if "embedContent" in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [ ]:
#Exemplo de embeddings. Define o título e o texto que serão usados para gerar os embeddings
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
      "\n"
      "Artigo completo:\n"
      "\n"
      "Gemini API & Google IA Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

#Gera os embeddings usando o modelo "models/embedding-001" para o texto e título fornecidos
embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type= "RETRIEVAL_DOCUMENT")
#Imprime os embeddings gerados
print(embeddings)

{'embedding': [0.057938386, -0.025222328, -0.029374618, 0.018128028, 0.07097077, -0.002999103, -0.034509502, -0.022444494, 0.059208263, 0.060102463, 0.012814384, 0.014694131, -0.052551787, -0.02563059, 0.012076601, -0.008769444, 0.027322203, -0.023831489, -0.029289097, -0.006141362, 0.003559453, 0.013716181, -0.038400453, -0.06677515, -0.014825566, 0.026962949, 0.008720244, -0.0425525, -0.0315799, 0.026684407, -0.050714113, 0.04651941, -0.034790684, 0.017373037, -0.045599148, -0.030061506, -0.03159216, -0.04994296, -0.0021664044, 0.0063544763, 0.003180075, -0.08144988, -0.013742884, 0.03210432, -0.0004508277, -0.026329117, 0.049961723, 0.04843151, 0.020552797, -0.056049462, 0.019889932, 0.021404311, 0.0569985, -0.044693258, -0.007124883, -0.0046869945, 0.019630777, -0.0375161, 0.017707013, -0.0094852485, 0.00873305, 0.024312833, -0.005869437, 0.054004494, 0.025829734, -0.06337846, -0.04069601, -0.00089311966, 0.00951865, 0.053325806, 0.00349321, 0.012829052, 0.050477892, -0.06732605, -

In [ ]:
#Listagem de documentos que serão buscados.Define três documentos com título e conteúdo que serão usados para busca
DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema"}
DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação"}
DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar marchas, basta mover a alavanca de câmbio para a posição desejada"}

#Cria uma lista com os documentos
documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [ ]:
#Cria um DataFrame do pandas a partir da lista de documentos
df = pd.DataFrame(documents)
#Define os nomes das colunas do DataFrame
df.columns = ["Título", "Conteúdo"]
#Imprime o DataFrame
df


,Título,Conteúdo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [ ]:
#Define o modelo que será usado para gerar os embeddings
model = "models/embedding-001"

In [ ]:
#Define uma função que gera embeddings para um dado título e texto
def embed_fn(title,text):
  #Retorna os embeddings gerados pelo modelo para o texto e título fornecidos
  return genai.embed_content(model=model,
                             content=text,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [ ]:
#Aplica a função embed_fn para cada linha do DataFrame, gerando embeddings para cada documento
df["embedding"] = df.apply(lambda row: embed_fn(row["Título"], row["Conteúdo"]), axis=1)
df

,Título,Conteúdo,embedding
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.0046640476, -0.018608006, -0.06823317, 0.0..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.016480235, -0.031649288, -0.046251215, -0.0..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[-0.006670459, -0.042065527, -0.03547557, 0.01..."


In [ ]:
#Define uma função que gera embeddings para uma consulta e busca o documento mais relevante na base de dados
def gerar_e_buscar_consulta(consulta, base, model):
  #Gera os embeddings para a consulta usando o modelo fornecido
  embedding_da_consulta = genai.embed_content(model=model,
                                              content=consulta,
                                              task_type="RETRIEVAL_QUERY")["embedding"]


  #Calcula o produto escalar entre os embeddings da consulta e os embeddings de cada documento na base de dados
  produtos_escalares = np.dot(np.stack(df["embedding"]), embedding_da_consulta)

  #Encontra o índice do documento com o maior produto escalar (documento mais similar à consulta)
  indice = np.argmax(produtos_escalares)
  #Retorna o conteúdo do documento mais similar à consulta
  return df.iloc[indice]["Conteúdo"]

In [ ]:
#Define a consulta que será usada para buscar na base de dados
consulta = "Como faço para trocar marchas em um carro do Google?"

#Busca o documento mais relevante para a consulta na base de dados
resposta = gerar_e_buscar_consulta(consulta, df, model)
#Imprime a resposta (conteúdo do documento mais relevante)
print(resposta)

Seu Googlecar tem uma transmissão automática. Para trocar marchas, basta mover a alavanca de câmbio para a posição desejada


In [ ]:
#Define as configurações de geração de texto, como temperatura e número de candidatos
generation_config = {
    "temperature": 0.5,
    "candidate_count": 1
}

In [ ]:
#Define o prompt para o modelo de geração de texto, incluindo a resposta da busca anterior
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {resposta}"
#Define o modelo de geração de texto que será usado
model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
#Gera o texto usando o prompt definido
response = model_2.generate_content(prompt)
#Imprime o texto gerado
print(response.text)

Ei, seu Googlecar maneiro! Ele tem uma transmissão automática, então você não precisa se preocupar em trocar marchas. Só precisa dar uma cutucadinha na alavanca de câmbio e pronto, marcha trocada!
